<a href="https://colab.research.google.com/github/Amir-D-Shadow/Google-Colab/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
\from google.colab import drive
#drive.flush_and_unmount()
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import plot_model
from numba import jit
import random
import os
import pandas as pd
import json
import cv2
import tensorflow.keras.backend as K
import math

In [3]:
def preprocess_class(input_path,save_path ,name="class_map.txt"):

   """
   MS COCO 2017 Dataset

   return dict
   """
   dataset = pd.read_csv(input_path)

   #get class map
   class_array = dataset.iloc[:,3]
   class_map = {}
   idx = 0
   
   for i in range(class_array.shape[0]):

      if not (class_array[i] in class_map.keys()):

         class_map[class_array[i]] = idx
         idx = idx + 1

   #save class map 
   with open(f"{save_path}/{name}","w") as file:
      
     file.write(json.dumps(class_map))

   return class_map


def preprocessing_label(input_path,save_path,name="gt_dataset.txt"):

   """
   save dict as {obj1:[[class,xmin,ymin,xcenter,ycenter],[class,xmin,ymin,xcenter,ycenter],...],obj2:...} (for each key)
   """

   #read csv file
   dataset = pd.read_csv(input_path)
   m = dataset.shape[0]

   #calibrate bbox pos
   dataset["xmin"] = dataset["xmin"] + (640 - dataset["width"] )//2
   dataset["xmax"] = dataset["xmax"] + (640 - dataset["width"] )//2

   dataset["ymin"] = dataset["ymin"] + (640 - dataset["height"] )//2
   dataset["ymax"] = dataset["ymax"] + (640 - dataset["height"] )//2

   #calculate center
   dataset["xcenter"] = (dataset["xmin"] + dataset["xmax"])/2
   dataset["ycenter"] = (dataset["ymin"] + dataset["ymax"])/2

   #Group the label
   gt_dataset = {}
   
   for i in range(m):

      filename = dataset.iloc[i,0]
      tmp = []

      #class
      tmp.append(dataset.iloc[i,3])

      #xmin xmax ymin ymax
      tmp.append(dataset.iloc[i,4].item())
      tmp.append(dataset.iloc[i,5].item())
      tmp.append(dataset.iloc[i,8].item())
      tmp.append(dataset.iloc[i,9].item())

      if not ( filename in gt_dataset.keys()  ):

         gt_dataset[filename] = []
         
      gt_dataset[filename].append(tmp)


   #save dataset
   with open(f"{save_path}/{name}","w") as file:

      file.write(json.dumps(gt_dataset))

      file.close()

   return gt_dataset
   
def preprocess_image(img,standard_shape=(640,640)):

   """
   img -- numpy.ndarray
   standard_shape -- (height,width)
   """
   #get h,w
   height,width = standard_shape

   #get pad size
   padH = (height - img.shape[0])//2
   padW = (width - img.shape[1])//2

   #pad img
   diff_H = height - img.shape[0]
   diff_W = width - img.shape[1]
   
   if (diff_H % 2 ) == 0 and (diff_W % 2) == 0:

      img_pad = np.pad(img,((padH,padH),(padW,padW),(0,0)),mode="constant",constant_values=(0,0))

   elif (diff_H % 2 ) != 0 and (diff_W % 2) == 0:

      img_pad = np.pad(img,((padH,padH+1),(padW,padW),(0,0)),mode="constant",constant_values=(0,0))

   elif (diff_H % 2 ) == 0 and (diff_W % 2) != 0:

      img_pad = np.pad(img,((padH,padH),(padW,padW+1),(0,0)),mode="constant",constant_values=(0,0))

   elif (diff_H % 2 ) != 0 and (diff_W % 2) != 0:

      img_pad = np.pad(img,((padH,padH+1),(padW,padW+1),(0,0)),mode="constant",constant_values=(0,0))

   return img_pad

In [4]:
def Kmean_IOU(bbox_hw,K=3,threshold=1e-8,max_iterations=2000):

   """
   bbox_hw -- (m,2) : h -> 0 , w -> 1

   return -- [large,medium ,small] <-- list(list)
   """

   m = bbox_hw.shape[0]

   #ONLY for K = 3 
   if m == 1:

      return [ [0] , [] , [] ]

   elif m == 2:

      bbox_1_area = bbox_hw[0,0] * bbox_hw[0,1]
      bbox_2_area = bbox_hw[1,0] * bbox_hw[1,1]

      if bbox_1_area > bbox_2_area:
         
         return [ [0] , [1] , [] ]

      else:

         return [ [1] , [0] , [] ]

   #initialize anchors (K,2)
   anchors = np.zeros((K,2))
   for i in range(K):

      anchors[i,:] = bbox_hw[i,:].copy()
   
   #calculate Kmean
   iteration_i = 0
   
   while iteration_i <= max_iterations:

      #store the data by index
      output_list = [[] for i in range(K)]

      #classify bbox 
      for i in range(m):

         class_id = best_anchor(bbox_hw[i,:].reshape(1,2),anchors)

         output_list[class_id].append(i)

      #update anchor box
      sum_diff = 0
      
      for i in range(K):
         
         new_h,new_w = update_anchor_x(bbox_hw,output_list[i],anchors[i,:].reshape(1,2))

         """
         #sum the changes for h and w of new anchor box 
         sum_diff = sum_diff + ((new_h-anchors[i,0])**2 + (new_w - anchors[i,1])**2)**(0.5)
         """
         #sum the changes for h and w of new anchor box - compare iou
         min_h = min(new_h,anchors[i,0])
         min_w = min(new_w,anchors[i,1])

         intersection = min_h * min_w
         union = new_h * new_w + anchors[i,0] * anchors[i,1] - intersection

         sum_diff = sum_diff + (1 - intersection / (union + 1e-8) )

         #update anchor box
         anchors[i,0] = new_h
         anchors[i,1] = new_w

      if sum_diff < threshold:

         return rearrange_output_list_to_correct_order(bbox_hw,output_list)

      #update iteration
      iteration_i = iteration_i + 1
      
   return rearrange_output_list_to_correct_order(bbox_hw,output_list)
         

def rearrange_output_list_to_correct_order(bbox_hw,output_list):

   #corrected output_list
   corrected_output_list = []
   
   n =len(output_list)
   #find area sum
   sample_area_sum = []
   for i in range(n):

      area = find_area_sum(bbox_hw,output_list[i])
      sample_area_sum.append((i,area))

   #sort the list
   sample_area_sum.sort(key=lambda x:x[1],reverse=True)

   #correct output list
   for ele in sample_area_sum:

      corrected_output_list.append(output_list[ele[0]])

   return corrected_output_list
   

def find_area_sum(bbox_hw,output_list_x):

   area = 0

   for i in output_list_x:

      area = area + bbox_hw[i,0] * bbox_hw[i,1]

   return area
      
   
def update_anchor_x(bbox_hw,output_list_x,anchors_x):

   """
   output_list_x -- [ n elements]
   anchor_x -- (1,2) : h -> 0 , w -> 1
   """

   sum_w = 0
   sum_h = 0

   n = len(output_list_x)

   for i in output_list_x:

      sum_h = sum_h + bbox_hw[i,0]
      sum_w = sum_w + bbox_hw[i,1]

   if n == 0:

      return anchors_x[0,0],anchors_x[0,1]

   mean_h = sum_h/n
   mean_w = sum_w/n

   return mean_h,mean_w


@jit(nopython=True)
def best_anchor(box,anchors):
   
  """
  box -- (1,2) :h -> 0 , w -> 1

  anchors -- (K,2)

  return class_idx
  """
  
  max_iou = 0
  max_index = 0

  K = anchors.shape[0]

  for i in range(K):

    min_h = np.minimum(box[0,0],anchors[i,0])
    min_w = np.minimum(box[0,1],anchors[i,1])

    intersection_area = min_h * min_w

    union_area = box[0,0] * box[0,1] + anchors[i,0] * anchors[i,1] - intersection_area

    cur_iou = intersection_area / (union_area + 1e-8)

    if cur_iou > max_iou:

      max_iou = cur_iou

      max_index = i

    
  return max_index

In [41]:
#generator
def get_gt_data(batch_size,img_info,class_info,img_path,img_shape = (640,640),aug_flag=False):

   """
   #img_shape -- (height,width)
   """
   
   img_list_shuffled = list(img_info.keys())
   
   random.shuffle(img_list_shuffled)

   m = len(img_list_shuffled)

   idx = 0

   while m >= batch_size:

      """
      #check remaining sample
      #if m < batch_size:

         #break
      """
      
      #get name list
      name_list = []

      for i in range(idx*batch_size,(idx+1)*batch_size):

         name_list.append(img_list_shuffled[i])

      #get image data -- np.array
      img_data = get_image_data(name_list,img_path,img_shape,aug_flag)

      #get y_true data -- tuple (np.array,np.array,np.array)
      label = get_y_true(img_data,name_list,img_info,class_info,img_shape)

      #update remaining sample
      m = m - batch_size
      idx = idx + 1

      yield img_data,label


      
def get_image_data(name_list,img_path,img_shape=(640,640),aug_flag=False):

   """
   return numpy.ndarray
   """

   img_data = []

   for name in name_list:

      #img -- numpy.ndarray
      img = cv2.imread(f"{img_path}/{name}")

      #calibrate image
      img = preprocess_image(img,img_shape)

      #img = cv2.resize(img,(128,128))
      #data augmentation
      if aug_flag:

         img = data_augment.data_aug(img)

      #save img
      img_data.append(img)

   img_data = np.array(img_data)


   return img_data


def get_y_true(img_data,name_list,img_info,class_info,img_shape = (640,640)):

   """
   name_list -- list
   img_info -- dict -- {obj1:[[class,xmin,ymin,xcenter,ycenter],[class,xmin,ymin,xcenter,ycenter],...],obj2:...} (for each key)
   class_info -- dict
   standard_scale -- dict (small , medium , large)
   img_shape -- (height,width)
   """
   #initialize y_true
   small_true = []
   medium_true = []
   large_true = []
   
   j = 0

   old_img = 0
   img = np.zeros((640,640,3))
   
   for name in name_list:

      img = img_data[j]

      #initialize y_true extra dim will be removed when it is saved (it is used for overlap region checking)
      obj_small_true = np.zeros((80,80,85))
      obj_medium_true = np.zeros((40,40,85))
      obj_large_true = np.zeros((20,20,85))

      #get (obj_info -- list)
      obj_info = img_info[name]

      n = len(obj_info)

      #initial bbox hw
      bbox_hw = np.zeros((n,2))

      #loop via all object in the image (obj -- list) to fill bbox_hw
      for i in range(n):

         bbox_hw[i,0] = (obj_info[i][4] - obj_info[i][2])*2
         bbox_hw[i,1] = (obj_info[i][3] - obj_info[i][1])*2

      #get cluster index
      cluster_idx = Kmean_IOU(bbox_hw)
      
      #update y_true
      obj_small_true,obj_medium_true,obj_large_true = update_y_true(img,obj_info,class_info,cluster_idx,obj_small_true,obj_medium_true,obj_large_true,img_shape = (640,640))
      
      #save image info
      small_true.append(obj_small_true[:,:,:])
      medium_true.append(obj_medium_true[:,:,:])
      large_true.append(obj_large_true[:,:,:])

      cv2.imwrite(f"{os.getcwd()}/gdrive/MyDrive/result/{name}",img)
      j = j + 1

   #convert y_true to numpy array
   small_true = np.array(small_true)
   medium_true = np.array(medium_true)
   large_true = np.array(large_true)

   return (large_true,medium_true,small_true)

   
def update_y_true(img,obj_info,class_info,cluster_idx,obj_small_true,obj_medium_true,obj_large_true,img_shape = (640,640)):

   """
   obj -- list [class,xmin,ymin,xcenter,ycenter]
   img_shape -- (height,width)
   """
   """
   _,xmin,ymin,xcenter,ycenter = obj

   #avoid x,y division by zeros for ratio calculation
   xmin = xmin + 1e-18
   ymin = ymin + 1e-18
   xcenter = xcenter + 1e-18
   ycenter = ycenter + 1e-18

   width = (xcenter - xmin) * 2
   height = (ycenter - ymin) * 2

   xmax = xmin + width
   ymax = ymin + height

   area  = width * height
   """
   #update large obj
   for i in cluster_idx[0]:

      #get obj info
      class_name,xmin,ymin,xcenter,ycenter = obj_info[i]

      class_id = class_info[class_name]

      #set up
      step_h = obj_large_true.shape[0] / img_shape[0]
      step_w = obj_large_true.shape[1] / img_shape[1]

      h_pos = int(step_h * ycenter)
      w_pos = int(step_w * xcenter)

      xmax = (xcenter - xmin)*2 + xmin
      ymax = (ycenter - ymin)*2 + ymin

      #draw
      img = cv2.circle(img,(int(xcenter),int(ycenter)),5,(255,0,0),-1)
      img = cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (255,0,0), 5)
      
      #prob
      obj_large_true[h_pos,w_pos,0] = 1

      #xmin,ymin
      obj_large_true[h_pos,w_pos,1] = xmin 
      obj_large_true[h_pos,w_pos,2] = ymin 

      #xcenter,ycenter
      obj_large_true[h_pos,w_pos,3] = xcenter 
      obj_large_true[h_pos,w_pos,4] = ycenter 

      #class
      obj_large_true[h_pos,w_pos,5+class_id] = 1

      #multiple positive
      obj_large_true = multiple_positive_labeling(img,obj_large_true,class_id,xmin,ymin,xmax,ymax,xcenter,ycenter,step_w,step_h)


   #update medium obj
   for i in cluster_idx[1]:

      #get obj info
      class_name,xmin,ymin,xcenter,ycenter = obj_info[i]

      class_id = class_info[class_name]

      #set up
      step_h = obj_medium_true.shape[0] / img_shape[0]
      step_w = obj_medium_true.shape[1] / img_shape[1]

      h_pos = int(step_h * ycenter)
      w_pos = int(step_w * xcenter) 

      xmax = (xcenter - xmin)*2 + xmin
      ymax = (ycenter - ymin)*2 + ymin

      #draw
      img = cv2.circle(img,(int(xcenter),int(ycenter)),5,(0,255,0),-1)
      img = cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,255,0), 5)
         
      #prob
      obj_medium_true[h_pos,w_pos,0] = 1

      #xmin,ymin
      obj_medium_true[h_pos,w_pos,1] = xmin 
      obj_medium_true[h_pos,w_pos,2] = ymin 

      #xcenter,ycenter
      obj_medium_true[h_pos,w_pos,3] = xcenter 
      obj_medium_true[h_pos,w_pos,4] = ycenter 

      #class
      obj_medium_true[h_pos,w_pos,5+class_id] = 1

      #multiple positive
      obj_medium_true = multiple_positive_labeling(img,obj_medium_true,class_id,xmin,ymin,xmax,ymax,xcenter,ycenter,step_w,step_h)


   #update small obj
   for i in cluster_idx[2]:

      #get obj info
      class_name,xmin,ymin,xcenter,ycenter = obj_info[i]

      class_id = class_info[class_name]

      #set up
      step_h = obj_small_true.shape[0] / img_shape[0]
      step_w = obj_small_true.shape[1] / img_shape[1]

      h_pos = int(step_h * ycenter)
      w_pos = int(step_w * xcenter)

      xmax = (xcenter - xmin)*2 + xmin
      ymax = (ycenter - ymin)*2 + ymin

      #draw
      img = cv2.circle(img,(int(xcenter),int(ycenter)),5,(0,0,255),-1)
      img = cv2.rectangle(img, (int(xmin),int(ymin)), (int(xmax),int(ymax)), (0,0,255), 5)
      
      #prob
      obj_small_true[h_pos,w_pos,0] = 1

      #xmin,ymin
      obj_small_true[h_pos,w_pos,1] = xmin 
      obj_small_true[h_pos,w_pos,2] = ymin 

      #xcenter,ycenter
      obj_small_true[h_pos,w_pos,3] = xcenter 
      obj_small_true[h_pos,w_pos,4] = ycenter 

      #class
      obj_small_true[h_pos,w_pos,5+class_id] = 1

      #multiple positive
      obj_small_true = multiple_positive_labeling(img,obj_small_true,class_id,xmin,ymin,xmax,ymax,xcenter,ycenter,step_w,step_h)
                 
   return obj_small_true,obj_medium_true,obj_large_true

             
def multiple_positive_labeling(img,y_true,class_id,xmin,ymin,xmax,ymax,xcenter,ycenter,step_w,step_h):

   """
   y_true -- numpy array
   """

   xlow = int(xmin*step_w)
   ylow = int(ymin*step_h)

   xhigh = int(xmax*step_w)
   yhigh = int(ymax*step_h)
   
   w_pos_init = int(xcenter*step_w - 32*step_w)
   h_pos_init = int(ycenter*step_h - 32*step_h)

   w_max = int(w_pos_init + 3*32*step_w)
   h_max = int(h_pos_init + 3*32*step_h)

   for w_pos in range(w_pos_init,w_max):

      for h_pos in range(h_pos_init,h_max):

         if (y_true[h_pos,w_pos,0] == 0) and (w_pos > xlow) and (w_pos < xhigh) and (h_pos > ylow) and (h_pos < yhigh):

            img = cv2.circle(img,(int(w_pos/step_w),int(h_pos/step_h)),5,(255,0,122),-1)

            #prob
            y_true[h_pos,w_pos,0] = 1

            #xmin,ymin
            y_true[h_pos,w_pos,1] = xmin + y_true[h_pos,w_pos,1] 
            y_true[h_pos,w_pos,2] = ymin + y_true[h_pos,w_pos,2]

            #xcenter,ycenter
            y_true[h_pos,w_pos,3] = xcenter + y_true[h_pos,w_pos,3]
            y_true[h_pos,w_pos,4] = ycenter + y_true[h_pos,w_pos,4]

            #class
            y_true[h_pos,w_pos,5+class_id] = 1


   return y_true

In [42]:
input_path = f"{os.getcwd()}/gdrive/MyDrive/annotations/test_annotations.csv"
save_path = f"{os.getcwd()}/gdrive/MyDrive/data"
img_train_info = preprocessing_label(input_path,save_path)

input_path = f"{os.getcwd()}/gdrive/MyDrive/annotations/train_annotations.csv"
save_path = f"{os.getcwd()}/gdrive/MyDrive/data"
class_train_info = preprocess_class(input_path,save_path)

img,label = next(get_gt_data(25,img_train_info,class_train_info,f"{os.getcwd()}/gdrive/MyDrive/image"))